In [1]:
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re

In [3]:
data = pd.read_excel('moviedata.xlsx')

In [19]:
list(data['영화명'])[0]

'카오산 탱고'

In [4]:
#query_txt = list(data['영화명'])
start_date="20100101"
end_date="20200718"
#q = '"영화"'

In [5]:
path = 'chromedriver.exe'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
# options.add_argument('disable-gpu')    # GPU 사용 안함
# options.add_argument('lang=ko_KR')    # 언어 설정, options=options
# driver = webdriver.Chrome('chromedriver', options=options)



driver = webdriver.Chrome(path)#, options=options)
driver.get('http://www.naver.com')
time.sleep(2)
# 네이버 검색창에 검색어 검색
element = driver.find_element_by_id('query')
element.send_keys("영화 ", list(data['영화명'])[0])  # query_txt는 위에서 입력한 '영화 리뷰'
element.submit()

# 블로그 카테고리 선택
driver.find_element_by_link_text('블로그').click()

# 검색옵션 버튼 클릭

driver.find_element_by_id('_search_option_btn').click()
    
#날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

#날짜 입력
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear()  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
    
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)

# 종료 날짜 입력
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)

driver.find_element_by_class_name('tx').click()
time.sleep(3)
    
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click()
time.sleep(1)
    
    
url_list = []
title_list = []

# 몇개의 페이지를 크롤링 할지 선택

total_page = 1
for j in tqdm(list(data['영화명'])[0:1]) : 
    for i in tqdm(range(total_page)) : # 페이지 번호
        i = i * 1 
        url = "https://search.naver.com/search.naver?date_from={0}&date_option=8&date_to={1}&dup_remove=1&nso=p%3Afrom{2}to{3}&post_blogurl=&post_blogurl_without=&query=영화%20{4}&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date, j, i)
   
        driver.get(url)
        time.sleep(0.5)
        
 # URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    moive_raw = driver.find_elements_by_css_selector(titles)

    # url 크롤링 시작    
    for movie in moive_raw:
        url = movie.get_attribute('href')   
        url_list.append(url)
    
     # 제목 크롤링 시작    
for i in list(data['영화명']) :
    for j in range(1, 11) :
        title_list.append(i)


print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))



df = pd.DataFrame({'url':url_list, 'title':title_list[:10]})

# 저장하기
df.to_excel("blog_url.xlsx")


url_load = pd.read_excel("blog_url.xlsx", index_col=0)        # 기본 모델
num_list = len(url_load)
number = 10
    
# 내용 크롤링 시작
content_list = []
for i in tqdm(range(0, number)) : 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
     # 내용 크롤링
    target_info = {}
    overlays = ".se-component.se-text.se-l-default"                                 
    contents = driver.find_elements_by_css_selector(overlays)         # date
    content_str = ' '.join(content_list)
        
    for content in contents:
         content_list.append(content.text)
    content_str = ' '.join(content_list)
    
    target_info['content'] = content_str

     # 글 하나 크롤링 후 크롬 창 닫기
    driver.close()       
    

target_info        
#df = pd.DataFrame({'url':url_list, 'title':title_list[:10], 'content':content_list})



url갯수:  10
title갯수:  82380


{'content': ''}

# 리뷰 크롤링

In [9]:
df

,url,title
0,https://blog.naver.com/antaregu?Redirect=Log&l...,카오산 탱고
1,https://blog.naver.com/cinemaplus?Redirect=Log...,카오산 탱고
2,https://blog.naver.com/ahnaa?Redirect=Log&logN...,카오산 탱고
3,https://blog.naver.com/kkcine?Redirect=Log&log...,카오산 탱고
4,https://blog.naver.com/actorpia?Redirect=Log&l...,카오산 탱고
5,https://blog.naver.com/boss4388?Redirect=Log&l...,카오산 탱고
6,https://pslove1207.blog.me/221269318767,카오산 탱고
7,https://blog.naver.com/cathy5320?Redirect=Log&...,카오산 탱고
8,https://blog.naver.com/kimgahyun-87?Redirect=L...,카오산 탱고
9,https://blog.naver.com/mylucky1029?Redirect=Lo...,카오산 탱고


In [274]:
# 여기서부터 리뷰 크롤링

url_load = pd.read_excel("blog_url.xlsx", index_col=0)        # 기본 모델

num_list = len(url_load)

print(num_list)


# ★수집할 글 갯수
#number = url_list
for i in tqdm(range(0, num_list)) : 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    

     # iframe 접근
    driver.switch_to_frame('mainFrame')

    # 내용 크롤링
    overlays = ".se-component.se-text.se-l-default"                                 
    contents = driver.find_elements_by_css_selector(overlays)         # date

    content_list = []
    for content in contents:
        content_list.append(content.text)

    content_str = ' '.join(content_list)

    # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
    target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
    dict[i] = target_info
    time.sleep(1)
        
    print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
    driver.close()       
    
    
    # 중간,중간에 파일로 저장하기
#     if i == 30 or 50 or 80:
#         # 판다스로 만들기
#         import pandas as pd
#         result_df = pd.DataFrame.from_dict(dict, 'index')

#         # 저장하기
#         result_df.to_excel("blog_content_movie.xlsx")
#         time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

# # 판다스로 만들기
# import pandas as pd
# result_df = pd.DataFrame.from_dict(dict, 'index')

# result_df.to_excel("blog_content_movie.xlsx")
# df2 = pd.read_excel('blog_content_movie.xlsx', index_col=0)

10


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.frame instead


0 카오산 탱고,2020
1 카오산 탱고,2020
2 카오산 탱고,2020
3 카오산 탱고,2020
4 카오산 탱고,2020
5 카오산 탱고,2020
6 카오산 탱고,2020
7 카오산 탱고,2020
8 카오산 탱고,2020
9 카오산 탱고,2020

수집한 글 갯수:  11
{'고': {'title': '고', 'content': '갑작스레 코로나가 창궐하여 끝을 알 수 없는 코비드19 시대가.\n계속되고 있다.\n더운 날씨에도 마스크를 달고 사는 것이 의무가 되어 버렸다.\n여러모로 불편하고 막막한 일들이 생겨버렸다.\n뭐 어쩌겠는가...\n하지만 그래도 기차는 달려야 하지 않나~\n예전에 즐겁게, 우여곡절 끝에 만들었던 독립영화 "카오산 탱고"가 개봉을 한다고 한다.\n개봉하기까지 감독이 애를 많이 썼다.\n물론 난 감독에게 이런저런 잔소리를 쏟아 냈다.\n꿋꿋하게 웃으며 나의 잔소리를 빨아들이던 감독의 모습이 선하다.\n태국에 여러 번 갔었지만\n그래도 김감독덕에 오랜 시간 태국에 머물 렀다.\n늦었지만 이렇게라도 고맙다는 말을 전한다.\n일도 하고 중간중간 즐거운 시간을 보네고 경험했던 \n기억들 떠오른다.\n우연히 만났던 좋은 사람들,\n물총난리,\n노곤한 발 마사지,\n거리의 흥겨운 음악,\n달콤한 과일,\n내가 좋아하는 소소한 음식들, \n혼자 떠나는 여행 등등..\n그 당시엔 모르지만 지나고 나면 찬란한 법이다.\n다들 해외여행도 가지 못하는데 이 영화 보며 잠깐 이나마 \n카오산에 다녀오길 바란다.\n물론 사회적 거리 두기이니 꼭 극장에서 보란 말은 못 하겠다.\n감독은 아쉬워하겠지만...\n각자 여력 것 챙겨 봤으면 하는 마음이다.\n태국은 참 아름답고 신비한 나라다. '}, 0: {'content': ''}, 1: {'content': ''}, 2: {'content': ''}, 3: {'content': ''}, 4: {'content': ''}, 5: {'content': ''}, 6: {'content': ''}, 7: {'conte

In [271]:
df2

,title,content
고,고,갑작스레 코로나가 창궐하여 끝을 알 수 없는 코비드19 시대가.\n계속되고 있다.\...


In [62]:
a = ['b', 'c']
'b' in a

True

In [64]:
a = '[영화] 해외 여행이 위험한 "우한 코로나" 의 시대....  그리운 태국 여행 대신에, 올 여름 보고 싶은 영화, "카오산 탱고"_20200709'

'카오산 탱고' in a

True

In [243]:
moive_raw2 = []

In [244]:
for i in list(data['영화명']) :
    for j in range(1, 11) :
        moive_raw2.append(i)

In [254]:
len(moive_raw2[:20])

20